In [53]:
# Imports
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
import cufflinks as cf
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
from keras.layers import MaxPooling1D, Conv1D, Dropout
##

ModuleNotFoundError: No module named 'cufflinks'

In [49]:
def read_data(dir):
    data_list = []
    for filename in os.listdir(dir):
        if filename.endswith(".xlsx"):
            data_list.append(pd.read_excel(os.path.join(dir, filename)))
        else:
            data_list.append(pd.read_csv(os.path.join(dir, filename), sep=';', encoding='latin-1', on_bad_lines='skip'))
    return data_list

In [34]:
#data = read_data("./data/clean_data/Håkvik")
#data = read_data("./data/clean_data/Nedre Neset")
data = read_data("./data/raw_data/Lyse/cascaded_use_case_data.xlsx")
target_Haavik = data[0]
target_Haavik.index = pd.to_datetime(target_Haavik.index)
target_Haavik.info()

NameError: name 'read_data' is not defined

In [35]:
scaler = MinMaxScaler()
target_data = target_Haavik.values
scaler.fit(target_data)
scaled_data = scaler.transform(target_data)

NameError: name 'target_Haavik' is not defined

In [36]:
def prepare_data(timeseries_data, n):
    X, y = [], []
    for i in range(len(timeseries_data)):
        end_idx = i + n
        if end_idx > len(timeseries_data) - 1:
            break
        seq_x, seq_y = timeseries_data[i:end_idx], timeseries_data[end_idx]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
n_features = 3
X, y = prepare_data(scaled_data, n_features)

NameError: name 'scaled_data' is not defined

In [37]:
# 70 / 20 / 10
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=int(len(target_Haavik)*0.3), shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=int(len(X_test_val)*0.3), shuffle=False)
X_train.shape, X_val.shape, X_test.shape

NameError: name 'X' is not defined

In [38]:
lstm = Sequential()
lstm.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(n_features, 1)))
lstm.add(LSTM(64, activation='relu'))
lstm.add(Dense(1))
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train, y_train, epochs=20, verbose=0, batch_size=64, validation_data=(X_val, y_val))
lstm.summary()

NameError: name 'Sequential' is not defined

In [39]:
cnn = Sequential()
cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_features, 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(50, activation='relu'))
cnn.add(Dense(1))
cnn.compile(optimizer='adam', loss='mse')
cnn.fit(X_train, y_train, epochs=20, verbose=0, batch_size=64, validation_data=(X_val, y_val))
cnn.summary()

NameError: name 'Sequential' is not defined

In [ ]:
mlp = Sequential()
mlp.add(Dense(64, activation='relu', input_dim=3))
mlp.add(Dense(1))
mlp.compile(optimizer='adam', loss='mse')
mlp.fit(X_train, y_train, epochs=20, verbose=0, batch_size=64, validation_data=(X_val, y_val))
mlp.summary()

In [ ]:
y_val = scaler.inverse_transform(y_val)

In [ ]:
lstm_predictions = lstm.predict(X_val).flatten()
lstm_predictions = scaler.inverse_transform(lstm_predictions.reshape(-1,1))
lstm_val_mae = mean_absolute_error(y_val, lstm_predictions)

In [ ]:
cnn_predictions = cnn.predict(X_val).flatten()
cnn_predictions = scaler.inverse_transform(cnn_predictions.reshape(-1,1))
cnn_val_mae = mean_absolute_error(y_val, cnn_predictions)

In [ ]:
mlp_predictions = mlp.predict(X_val).flatten()
mlp_predictions = scaler.inverse_transform(mlp_predictions.reshape(-1,1))
mlp_val_mae = mean_absolute_error(y_val, mlp_predictions)

In [ ]:
idx_val = target_Haavik.index[len(X_train):len(X_train)+len(X_val)]

In [5]:
predictions = pd.DataFrame(data={
    'LSTM predictions': [y[0] for y in lstm_predictions],
    'Actuals': [y[0] for y in y_val]}, index=idx_val)
predictions.iplot(kind='spread')

NameError: name 'lstm_predictions' is not defined

In [6]:
mean_value = np.mean(target_Haavik.values)
val_comparisons = pd.DataFrame(data={
    'cnn': [cnn_val_mae, mean_squared_error(y_val, cnn_predictions, squared=False), mean_squared_error(y_val, cnn_predictions, squared=False)/ mean_value],
    'lstm': [lstm_val_mae, mean_squared_error(y_val, lstm_predictions, squared=False), mean_squared_error(y_val, lstm_predictions, squared=False)/ mean_value],
    'mlp': [mlp_val_mae, mean_squared_error(y_val, mlp_predictions, squared=False), mean_squared_error(y_val, mlp_predictions, squared=False)/ mean_value],
}, index=['MAE', 'RMSE', 'NRMSE'])
val_comparisons

NameError: name 'target_Haavik' is not defined

In [7]:
y_test = scaler.inverse_transform(y_test)
lstm_test_predictions = lstm.predict(X_test)
lstm_test_predictions = scaler.inverse_transform(lstm_test_predictions.reshape(-1,1))
lstm_test_mae = mean_absolute_error(y_test, lstm_test_predictions)
print("Mean Absolute Error lstm on test data:", lstm_test_mae)
print("RMSE on test data:", mean_squared_error(y_test, lstm_test_predictions, squared=False))
print("NRSME on test data", mean_squared_error(y_test, lstm_test_predictions, squared=False)/ mean_value)

NameError: name 'scaler' is not defined

In [ ]:
test_predictions = pd.DataFrame(data={
    'LSTM predictions': [y[0] for y in lstm_test_predictions],
    'Actuals': [y[0] for y in y_test]}, index=target_Haavik.index[len(target_data)-len(y_test):])
test_predictions.iplot(kind='spread')

In [ ]:
y_train = scaler.inverse_transform(y_train)
lstm_train_predictions = lstm.predict(X_train)
lstm_train_predictions = scaler.inverse_transform(lstm_train_predictions.reshape(-1,1))
lstm_test_mae = mean_absolute_error(y_train, lstm_train_predictions)
print("Mean Absolute Error lstm on train data:", lstm_test_mae)
print("RMSE on train data:", mean_squared_error(y_train, lstm_train_predictions, squared=False))
print("NRSME on train data", mean_squared_error(y_train, lstm_train_predictions, squared=False)/ mean_value)

In [ ]:
val = [x[0] for x in X_val]
history = [x for x in val]
p = list()
for i in range(len(val)):
	yhat = history[-1]
	p.append(yhat)
	obs = val[i]
	history.append(obs)
print("Mean Absolute Error:", mean_absolute_error(y_val, scaler.inverse_transform(p)))
print("RMSE:", mean_squared_error(y_val, scaler.inverse_transform(p), squared=False))